# 本チャプターの目次
1. オペレーショナルメタデータは5w1h
2. オペレーショナルメタデータの使い道は？
3. アクセスログを使って5w1hを管理してみよう
4. オペレーショナルメタデータの結果をメタデータストアに格納しよう
5. メタデータの形は最終的にどうなったのか？

# オペレーショナルメタデータは5w1h
オペレーショナルメタデータとは、データに関する5w1hを取得することです。  
どれくらいアクセスがあるのか？誰が主に使っているのか？
そんな情報を取得し、メタデータストアに格納します。

## 最もオーソドックスな方法はアクセスログを利用すること
そして、データに関する5w1hを取得するために利用される最もオーソドックスな形がアクセスログを用いた5w1hの取得になります。  
セクション２にて、環境構築をした際にlog4j.propertiesの設定を行なっているのですが、そこで今回利用する/var/log/spark/spark_operation.logを出力する設定を入れています。  

今回は、リポジトリ内に「loggiles/spark_operation.log.1」というフォルダを使って解析を進めますが、「/var/log/spark/」配下を直接みて実行いただいても問題ありません。

# オペレーショナルメタデータの使い道は？
オペレーショナルメタデータの役割の一つとして「データの沼化」を防ぐ効果があります。  
データの沼化とは、使われないデータでデータレイク（やデータウェアハウス）が埋め尽くされてしまうような状況を指します。  

そこでしっかりと、データに関するアクセスの状況を把握することによって、例えばアクセス数0が続くのであればデータを削除したりすることが検討できますし、  
元々誰がアクセスしていたかを記録にとっていれば、そのユーザに対して聞き込みをすすることもできることになります  


# アクセスログを使って5w1hを管理してみよう
それでは、早速ここからアクセスログを使ってオペレーショナルメタデータを取得していきましょう。

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init()

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。

In [ ]:
# 今回ターゲットとするのは、以下のようなログの内容です。
# @audit@の部分で監査ログであることがわかるのと、ugiでアクセスユーザ、ipで場所、cmdでどこのデータにアクセスしたのか、時間でいつアクセスしたのか？がわかります
# howにあたる部分は今回はSpark限定で、whyにあたる部分は他の情報を使うことによって明確になります(主には聞くことが多い)
# 21/11/23 11:27:53 @INFO @audit@ ugi=saitouyuuki	ip=unknown-ip-addr	cmd=get_database: default	
from pyspark.sql.functions import split, col
df=spark.read.text("/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/logfiles/spark_operation.log.1")

df_log=df.withColumn("tmp", split(df.value,"@")).select(
    col("tmp").getItem(0).alias('time'),
    col("tmp").getItem(1).alias('log_level'),
    col("tmp").getItem(2).alias('category'),
    col("tmp").getItem(3).alias('log'))

df_log.show()

In [ ]:
# いくつか不要な情報を除いていきましょう
df_log.filter("category = 'audit'").show(truncate=False)
df_log_f=df_log.filter("category = 'audit'")

In [ ]:
# ugi=saitouyuuki\tip=unknown-ip-addr\tcmd=get_database: data_management_crush_course\tの部分を
# \tでさらに分割すればさらにこまかい分析が出来そうですね。

df_log_split=df_log_f.withColumn("tmp", split(df_log_f.log,"\\t")).select(
    df_log_f.time,
    df_log_f.log_level,
    df_log_f.category,
    col("tmp").getItem(0).alias('ugi'),
    col("tmp").getItem(1).alias('ip'),
    col("tmp").getItem(2).alias('cmd'))

df_log_split.show()

In [ ]:
from pyspark.sql.functions import trim
# 21/11/23 11:27:55 |    INFO |   audit| ugi=saitouyuuki|ip=unknown-ip-addr|cmd=get_table : d
# あと一歩最後の:をsuplitしたいですね。

df_log_split_check=df_log_split.withColumn("tmp", split(df_log_split.cmd," : ")).select(
    df_log_split.time,
    df_log_split.log_level,
    df_log_split.category,
    df_log_split.ugi,
    df_log_split.ip,
    trim(col("tmp").getItem(0)).alias('cmd_sub'),
    col("tmp").getItem(1).alias('cmd_sub2')
    )

df_log_split_check.show()

In [ ]:
# 今回はこの中から、特定のテーブルへのアクセス数を数えてみようと思います
# 特定のテーブルへのアクセスを抜き出すためにはcmd-get_tableが良さそうです。
# また、時間的に重複しているようなものもあるのでそちらは除外しましょう

df_log_split_check=df_log_split_check.filter("cmd_sub='cmd=get_table'").distinct()
df_log_split_check.createOrReplaceTempView("original")

result=spark.sql("""
select '2021/11/12 10:00:00' as time,ugi , ip , cmd_sub2, cast(count(*) as String) as access
from 
original
group by ugi , ip , cmd_sub2
"""
)

result.show(truncate=False)

In [ ]:
# これで誰がどこから、どこに、何回アクセスしてるかがわかりました。
# アクセスの理由についてはアクセスしてきた人（不正理由でなければ）聞くことができます
# 今回は、jinko_tableのメタデーを取得していますので、「db=data_management_crush_course tbl=jinko_table」のアクセス数を取得します。

result_string=''
# あまり関係ないですが、集計した後のデータであれば以下のようにループを回すことが可能です
for x in result.filter("cmd_sub2='db=data_management_crush_course tbl=jinko_table'").collect():
    result_string=','.join(x)

# オペレーショナルメタデータの結果をメタデータストアに格納しよう
これで全ての作業が完了しました。

前のレクチャーで取得したアクセスログの結果をSparkテーブルへ登録していきましょう。


In [ ]:
from pyspark.sql.functions import when
metadata_df=spark.sql("select * from metadata_tmp.sample_metadata ")

# メタデータ取得対象のデータを更新する
metadata_df = metadata_df.withColumn("frequency_access", when(metadata_df.frequency_access.isNull() ,result_string).otherwise(metadata_df.frequency_access))
metadata_df.show(truncate=False)

In [ ]:
#取得したデータをmetadata_tmp.sample_metadataに格納していきます
#読み込んだテーブルに対して直接データを入れることができないので、一度ファイルを吐き出します

spark.sql("REFRESH TABLE metadata_tmp.sample_metadata")
metadata_df.write.mode('overwrite').parquet("/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/tmp/")
insert_df=spark.read.parquet("/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/tmp/")

#取得したデータをmetadata_tmp.sample_metadataに格納していきます
insert_df.createOrReplaceTempView("sample")

spark.sql("""
Insert overwrite  table metadata_tmp.sample_metadata 
select  * from sample
""")


In [ ]:
# 結果の確認をしてみます
spark.sql("select * from metadata_tmp.sample_metadata ").show(truncate=False)

# 無事入っているようですね
# 次のセクションでは、Sparkテーブルに保存した結果をMysqlテーブルに格納してみます。

# メタデータの形は最終的にどうなったのか？
まずは結果を確認する前に、Sparkテーブルの結果をMysqlに格納しましょう。　　
Mysqlに格納する理由は他のアプリケーション（APIやGUI）からの連携性を高めるためでした。

SparkテーブルとMysqlテーブルの大きな違いを上げるのであれば、実行速度の違いが挙げられます。  


In [ ]:
# JDBC経由で結果をMysqlに入れることが可能です
# JDBCの接続情報は

metadata_df=spark.sql("select * from metadata_tmp.sample_metadata ")

metadata_df.write.format('jdbc').options(
      url='jdbc:mysql://localhost/metadata?enabledTLSProtocols=TLSv1.2',
      driver='com.mysql.jdbc.Driver',
      dbtable='metadatas',
      user='root',
      password='root').mode('overwrite').save()

# Mysqlのコマンドです。

Mysqlに接続後、以下のコマンドを順番に発行してみてください。

```
use metadata;
select * from metadatas;
```

これらの値を、APIであったりGUIで表示することによってテーブル定義の管理であったり、データ基盤以外のシステムが稼働状況を把握できるようになってくるというわけです。

## サマリーのカラムについて
現状はサマリーのカラムは仮の文字を入れていました。  
この項目は、ビジネスユーザに更新してもらうことを想定しているカラムです(主にGUIなどから)  
テーブルの構造はわかっても、カラムの更新値がどのようなタイミングでその値に更新されるのか？は知っているようで知らない情報だったりします  
そのような暗黙知を形式知に変えていくためのカラムがサマリーカラムの役割です（実際はもっと複雑ですがまずはビジネスメタデータの役割などを理解してもらえると嬉しいです）

In [ ]:
spark.stop()
spark.sparkContext.stop()